In [ ]:
import fitz
from pdf2image import convert_from_path
import pytesseract
import cv2
import csv
import ssl
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import pipeline

In [ ]:
ssl._create_default_https_context = ssl._create_unverified_context # Disable SSL authentication

def find_rectangles(pdf_file_path, search_text):
    rectangles = []

    with fitz.open(pdf_file_path) as doc:
        for page_number in range(doc.page_count):
            page = doc[page_number]
            page_text = page.get_text()

            if search_text in page_text:
                pages = convert_from_path(pdf_file_path, dpi=300, first_page=page_number+1, last_page=page_number+1)

                img_path = f'{pdf_file_path}_page_{page_number+1}.png'
                pages[0].save(img_path, 'PNG')

                img = cv2.imread(img_path)
                # Ön işleme adımı eklendi
                img = cv2.medianBlur(img, 3)
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

                # Gerçekleştirilen OCR işlemi
                text = pytesseract.image_to_string(thresh, lang='eng', config='--psm 6')

                if search_text in text:
                    rectangles.append((page_number+1, 1, 0, 0, img.shape[1], img.shape[0]))

                    cv2.rectangle(img, (0, 0), (img.shape[1], img.shape[0]), (0, 0, 255), 2)
                    cv2.imwrite(f'page_{page_number+1}_rects.png', img)

                    # Hugging Face modeli kullanarak soruya yanıt verme
                    tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
                    model = AutoModelForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
                    nlp = pipeline('question-answering', model=model, tokenizer=tokenizer)

                    question = input("Enter a question: ") # Ask the user to enter a question
                                                           # Example question: What is the meaning of life?
                    result = nlp({
                        'question': question,  # This time use the question entered by the user
                        'context': text
                    })
                    answer = result['answer']
                    if answer:  # Write the coordinates to the CSV file if the answer is found
                        with open('coordinates.csv', mode='a', newline='') as csv_file:
                            fieldnames = ['Page Number', 'Rectangle Number', 'X', 'Y', 'Width', 'Height', 'Answer']
                            writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

                            writer.writerow({'Page Number': page_number+1,
                                              'Rectangle Number': 1,
                                              'X': 0,
                                              'Y': 0,
                                              'Width': img.shape[1],
                                              'Height': img.shape[0],
                                              'Answer': answer})

    return rectangles